<div style="color:rgb(0,0,255);font-size: 40px;font-weight:700;">
MAIN SETTINGS
</div>

In [ ]:
###SETTINGS###

import os
import re
import time
import subprocess
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed
from getpass import getpass
from urllib.parse import urlencode

# Platform detection
ON_KAGGLE = os.path.exists('/kaggle')
ON_COLAB = 'COLAB_RELEASE_TAG' in os.environ or os.path.exists('/content')
ON_VAST = any(k in os.environ for k in ("VAST_CONTAINERLABEL", "VAST_TCP_PORT_22", "CONTAINER_ID")) or os.path.exists('/workspace')


MAX_PARALLEL_DOWNLOADS = max(1, int(os.environ.get("MAX_PARALLEL_DOWNLOADS", "3")))
MIN_VALID_FILE_BYTES = int(os.environ.get("MIN_VALID_FILE_BYTES", "1000000"))

if shutil.which("aria2c") is None:
    print("aria2c not found → installing...")
    try:
        subprocess.run(["apt", "update", "-qq"], check=True, capture_output=True)
        result = subprocess.run(["apt", "install", "-y", "-qq", "aria2"], capture_output=True, text=True)
        if result.returncode == 0:
            print("aria2c installed successfully")
        else:
            print("Install failed (code {}):".format(result.returncode))
            print("stderr:", result.stderr.strip())
    except subprocess.CalledProcessError as e:
        print(f"apt error (code {e.returncode}): {e.stderr}")
    except Exception as e:
        print(f"Unexpected error: {e}")
else:
    print("aria2c already available")

# Determining the working directory
possible_bases = [
    "/workspace",       # Vast.ai / RunPod
    "/kaggle/working",  # Kaggle
    "/content",         # Google Colab
]

BASE_DIR = None
for path in possible_bases:
    if os.path.isdir(path):
        BASE_DIR = path
        break

if BASE_DIR is None:
    BASE_DIR = os.getcwd()
    print("WARNING: Known directory not found:", BASE_DIR)

print("Working directory:", BASE_DIR)

# Configuration
FORGE_DIR        = os.path.join(BASE_DIR, "stable-diffusion-webui-forge")
MODELS_DIR       = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Stable-diffusion")
LORA_DIR         = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Lora")
CONTROLNET_DIR   = os.path.join(FORGE_DIR, "extensions", "sd-webui-controlnet")
CONTROLNET_MODELS_DIR = os.path.join(CONTROLNET_DIR, "models")
EXTENSIONS_DIR   = os.path.join(FORGE_DIR, "extensions")
OUTPUTS_DIR      = os.path.join(FORGE_DIR, "outputs")

for d in [MODELS_DIR, LORA_DIR, CONTROLNET_DIR, CONTROLNET_MODELS_DIR, EXTENSIONS_DIR, OUTPUTS_DIR]:
    os.makedirs(d, exist_ok=True)


def get_secret(name: str):
    """Get secret from env/Kaggle/Colab only."""
    value = os.environ.get(name)
    if value:
        return value.strip(), "env"

    # Kaggle secrets
    if ON_KAGGLE:
        try:
            from kaggle_secrets import UserSecretsClient
            value = UserSecretsClient().get_secret(name)
            if value:
                return value.strip(), "kaggle_secrets"
        except Exception:
            pass

    # Colab secrets panel: from google.colab import userdata
    if ON_COLAB:
        try:
            from google.colab import userdata
            value = userdata.get(name)
            if value:
                return value.strip(), "colab_userdata"
        except Exception:
            pass

    return None, None


CIVITAI_TOKEN, CIVITAI_SRC = get_secret("CIVITAI_TOKEN")
HF_TOKEN, HF_SRC = get_secret("HF_TOKEN")

if not CIVITAI_TOKEN:
    manual_civitai = getpass("Enter CIVITAI_TOKEN (leave blank to skip): ").strip()
    if manual_civitai:
        CIVITAI_TOKEN, CIVITAI_SRC = manual_civitai, "manual_input"

if not HF_TOKEN:
    manual_hf = getpass("Enter HF_TOKEN (leave blank to skip): ").strip()
    if manual_hf:
        HF_TOKEN, HF_SRC = manual_hf, "manual_input"

TOKENS = {}
if CIVITAI_TOKEN:
    TOKENS["CIVITAI"] = CIVITAI_TOKEN
if HF_TOKEN:
    TOKENS["HF_TOKEN"] = HF_TOKEN

print("Token sources:")
print(f"  CIVITAI_TOKEN: {CIVITAI_SRC or 'not found'}")
print(f"  HF_TOKEN: {HF_SRC or 'not found'}")
if ON_VAST:
    print("Vast.ai tip: add CIVITAI_TOKEN/HF_TOKEN in template env vars, restart container, then rerun this cell.")

if not CIVITAI_TOKEN:
    print("CivitAI token not found")
if not HF_TOKEN:
    print("HF token not found")
if not TOKENS:
    raise RuntimeError("No tokens were provided. Set secrets or enter at least one token (CivitAI or HF).")


def _prepare_download_url(url, token):
    """CivitAI download works more reliably with token as query param."""
    if token and "civitai.com/api/download/models" in url and "token=" not in url:
        sep = "&" if "?" in url else "?"
        return f"{url}{sep}{urlencode({'token': token})}"
    return url


def _looks_valid_file(path, min_bytes=MIN_VALID_FILE_BYTES):
    return os.path.exists(path) and os.path.getsize(path) > min_bytes


def _human_mb(num_bytes):
    return f"{num_bytes / (1024 * 1024):.1f} MB"


def _estimate_expected_mb(label):
    # Examples: "151 MB", "6,46 GB"
    match = re.search(r"(\d+[\.,]?\d*)\s*(MB|GB)", label, re.IGNORECASE)
    if not match:
        return None
    value = float(match.group(1).replace(',', '.'))
    unit = match.group(2).upper()
    return value * (1024 if unit == "GB" else 1)


def _size_sanity_warning(path, expected_mb, tolerance=0.7):
    if expected_mb is None or not os.path.exists(path):
        return
    actual_mb = os.path.getsize(path) / (1024 * 1024)
    if actual_mb < expected_mb * tolerance:
        print(f"  WARNING: file size looks low ({actual_mb:.1f} MB vs expected ~{expected_mb:.1f} MB)")


def _has_min_free_disk(path, required_mb, reserve_mb=1024):
    if required_mb is None:
        return True
    usage = shutil.disk_usage(path)
    free_mb = usage.free / (1024 * 1024)
    return free_mb >= (required_mb + reserve_mb)


def _require_download_tools():
    found = [name for name in ("aria2c", "curl", "wget") if shutil.which(name)]
    if not found:
        raise RuntimeError("No download tools found. Install at least one of: aria2c, curl, wget")
    print(f"Download tools available: {', '.join(found)}")


def download_file(url, output_path, token=None, expected_mb=None):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    if _looks_valid_file(output_path):
        print(f"  Already ready → {_human_mb(os.path.getsize(output_path))}")
        return True

    if not _has_min_free_disk(os.path.dirname(output_path), expected_mb):
        raise RuntimeError(f"Not enough free disk for {os.path.basename(output_path)}")

    token = token.strip() if isinstance(token, str) else token
    final_url = _prepare_download_url(url, token)
    temp_output_path = f"{output_path}.part"

    attempts = []

    if shutil.which("aria2c"):
        aria2_cmd = [
            "aria2c", "-x", "16", "-s", "16", "-k", "1M",
            "--summary-interval=1", "--console-log-level=warn",
            "--max-tries=5", "--retry-wait=3", "--timeout=30", "--continue=true",
            "-d", os.path.dirname(output_path),
            "-o", os.path.basename(temp_output_path),
            final_url,
        ]
        if token:
            aria2_cmd += ["--header", f"Authorization: Bearer {token}"]
        attempts.append(("aria2c", aria2_cmd))

    if shutil.which("curl"):
        curl_cmd = [
            "curl", "-L", "-C", "-", "--fail", "--progress-bar",
            "--retry", "5", "--retry-delay", "3", "--connect-timeout", "20", "--max-time", "0",
            "-A", "Mozilla/5.0",
            final_url, "-o", temp_output_path,
        ]
        if token:
            curl_cmd = curl_cmd[:1] + ["-H", f"Authorization: Bearer {token}"] + curl_cmd[1:]
        attempts.append(("curl", curl_cmd))

    if shutil.which("wget"):
        wget_cmd = [
            "wget", "-c", final_url, "-O", temp_output_path,
            "--tries=5", "--waitretry=3", "--timeout=30", "--read-timeout=30",
            "--user-agent=Mozilla/5.0",
            "--progress=dot:giga",
        ]
        if token:
            wget_cmd = wget_cmd[:2] + ["--header", f"Authorization: Bearer {token}"] + wget_cmd[2:]
        attempts.append(("wget", wget_cmd))

    last_error = None
    for tool_name, cmd in attempts:
        try:
            start = time.time()
            print(f"  [{tool_name}] started")
            subprocess.run(cmd, check=True)
            elapsed = max(time.time() - start, 0.01)

            if not _looks_valid_file(temp_output_path):
                raise RuntimeError(f"{tool_name} finished but file is empty/small")

            os.replace(temp_output_path, output_path)
            size_bytes = os.path.getsize(output_path)
            speed_mbps = (size_bytes * 8) / elapsed / 1_000_000
            print(f"  OK → {os.path.basename(output_path)} ({tool_name}, {_human_mb(size_bytes)}, ~{speed_mbps:.1f} Mbit/s)")
            _size_sanity_warning(output_path, expected_mb)
            return True
        except Exception as e:
            last_error = e
            if os.path.exists(temp_output_path) and os.path.getsize(temp_output_path) < MIN_VALID_FILE_BYTES:
                try:
                    os.remove(temp_output_path)
                except Exception:
                    pass
            print(f"  {tool_name} failed: {e}")

    raise RuntimeError(f"All download methods failed for {output_path}. Last error: {last_error}")


def run_download_list(items, target_dir, list_name):
    """Parallel-safe downloader with compact final summary."""
    print(f"Total entries in {list_name}: {len(items)}")

    def _task(item):
        name, url, filename, source = item
        output_path = os.path.join(target_dir, filename)

        if _looks_valid_file(output_path):
            return name, filename, "skipped", None

        token = TOKENS.get(source)
        expected_mb = _estimate_expected_mb(name)

        try:
            print(f"  Loading {name:<35} ({source}) → {filename}")
            download_file(url=url, output_path=output_path, token=token, expected_mb=expected_mb)
            return name, filename, "ok", None
        except Exception as e:
            return name, filename, "failed", str(e)

    results = []
    with ThreadPoolExecutor(max_workers=MAX_PARALLEL_DOWNLOADS) as ex:
        futures = [ex.submit(_task, item) for item in items]
        for fut in as_completed(futures):
            results.append(fut.result())

    ok = sum(1 for _, _, status, _ in results if status == "ok")
    skipped = sum(1 for _, _, status, _ in results if status == "skipped")
    failed = [(n, f, err) for n, f, status, err in results if status == "failed"]

    print(f"\n{list_name} done. New: {ok}, skipped: {skipped}, failed: {len(failed)}")
    if failed:
        print("Failed entries:")
        for name, filename, err in failed:
            print(f"  - {name} ({filename}) -> {err}")


_require_download_tools()
print(f"MAX_PARALLEL_DOWNLOADS = {MAX_PARALLEL_DOWNLOADS}")



<div style="color:rgb(0,0,255);font-size: 40px;font-weight:700;">
DOWNLOAD BLOCK
</div>

In [ ]:
### CONTROLNET INSTALL OPTIONS ###

# Option A (default): force clean reinstall
# Option B: set CONTROLNET_INSTALL_MODE = "update" for git pull in existing repo
# Option C: set CONTROLNET_INSTALL_MODE = "skip" to keep current state

CONTROLNET_REPO_URL = "https://github.com/Mikubill/sd-webui-controlnet"
CONTROLNET_INSTALL_MODE = os.environ.get("CONTROLNET_INSTALL_MODE", "reinstall").strip().lower()

if CONTROLNET_INSTALL_MODE not in {"reinstall", "update", "skip"}:
    raise ValueError("CONTROLNET_INSTALL_MODE must be one of: reinstall, update, skip")

print(f"ControlNet extension path: {CONTROLNET_DIR}")
print(f"Install mode: {CONTROLNET_INSTALL_MODE}")

if CONTROLNET_INSTALL_MODE == "skip":
    print("ControlNet install skipped")
else:
    if os.path.isdir(CONTROLNET_DIR) and CONTROLNET_INSTALL_MODE == "reinstall":
        print("Removing existing ControlNet directory...")
        shutil.rmtree(CONTROLNET_DIR)

    if not os.path.isdir(CONTROLNET_DIR):
        print("Cloning ControlNet repository...")
        subprocess.run(["git", "clone", CONTROLNET_REPO_URL, CONTROLNET_DIR], check=True)
    else:
        print("Updating ControlNet repository...")
        subprocess.run(["git", "-C", CONTROLNET_DIR, "pull", "--ff-only"], check=True)

os.makedirs(CONTROLNET_MODELS_DIR, exist_ok=True)
print("ControlNet repository is ready")
print(f"ControlNet models directory: {CONTROLNET_MODELS_DIR}")

#ControlNET models download

controlnet_models_to_download = [
    ("t2i-adapter_xl_openpose 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors", "t2i-adapter_xl_openpose.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_canny 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors", "t2i-adapter_xl_canny.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_sketch 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors", "t2i-adapter_xl_sketch.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_midas 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_midas.safetensors", "t2i-adapter_diffusers_xl_depth_midas.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_zoe 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_zoe.safetensors", "t2i-adapter_diffusers_xl_depth_zoe.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_lineart 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors", "t2i-adapter_diffusers_xl_lineart.safetensors", "HF_TOKEN"),
]

run_download_list(controlnet_models_to_download, CONTROLNET_MODELS_DIR, "ControlNet")



#Checkpoints download

models_to_download = [
    ("WAI ILL V16.0 6,46 GB", "https://civitai.com/api/download/models/2514310?type=Model&format=SafeTensor&size=pruned&fp=fp16", "waiIllustriousSDXL_v160.safetensors", "CIVITAI"),
]

run_download_list(models_to_download, MODELS_DIR, "Checkpoints")



#LoRa download

lora_to_download = [
    ("Detailer IL V2 218 MB",        "https://civitai.com/api/download/models/1736373?type=Model&format=SafeTensor",    "detailer_v2_il.safetensors",     "CIVITAI"),
    ("Realistic filter V1 55 MB",    "https://civitai.com/api/download/models/1124771?type=Model&format=SafeTensor",    "realistic_filter_v1_il.safetensors", "CIVITAI"),
    ("Hyperrealistic V4 ILL 435 MB", "https://civitai.com/api/download/models/1914557?type=Model&format=SafeTensor",    "hyperrealistic_v4_ill.safetensors",  "CIVITAI"),
    ("Niji semi realism V3.5 ILL 435 MB", "https://civitai.com/api/download/models/1882710?type=Model&format=SafeTensor", "niji_semi_realism_v35.safetensors", "CIVITAI"),
    ("ATNR Style ILL V1.1 350 MB", "https://civitai.com/api/download/models/1711464?type=Model&format=SafeTensor", "atnr_style_ill_v1.1.safetensors", "CIVITAI"),
    ("Face Enhancer Ill 218 MB", "https://civitai.com/api/download/models/1839268?type=Model&format=SafeTensor", "face_enhancer_ill.safetensors", "CIVITAI"),
    ("Smooth Detailer Booster V4 243 MB", "https://civitai.com/api/download/models/2196453?type=Model&format=SafeTensor", "smooth_detailer_booster_v4.safetensors", "CIVITAI"),
    ("USNR Style V-pred 157 MB", "https://civitai.com/api/download/models/2555444?type=Model&format=SafeTensor", "usnr_style_v_pred.safetensors", "CIVITAI"),
    ("748cm Style V1 243 MB", "https://civitai.com/api/download/models/1056404?type=Model&format=SafeTensor", "748cm_style_v1.safetensors", "CIVITAI"),
    ("Velvet's Mythic Fantasy Styles IL 218 MB", "https://civitai.com/api/download/models/2620790?type=Model&format=SafeTensor", "velvets_mythic_fantasy_styles_il.safetensors", "CIVITAI"),
    ("Pixel Art Style IL V7 435 MB", "https://civitai.com/api/download/models/2396350?type=Model&format=SafeTensor", "Pixel Art Style IL", "CIVITAI"),
]

run_download_list(lora_to_download, LORA_DIR, "LoRA")



<div style="color:rgb(0,0,255);font-size: 40px;font-weight:700;">
ARCHIVE+OUTPUT
</div>

In [ ]:
# Archive outputs for download (folders like DD_MM_YYYY with PNG files)

import os
import shutil
from datetime import datetime

archive_source = OUTPUTS_DIR
stamp = datetime.now().strftime("%d_%m_%Y_%H%M")
archive_base = os.path.join(BASE_DIR, f"outputs_backup_{stamp}")
archive_zip = shutil.make_archive(archive_base, "zip", archive_source)

size_mb = os.path.getsize(archive_zip) / (1024 * 1024)
print(f"Archive created: {archive_zip}")
print(f"Archive size: {size_mb:.1f} MB")

if ON_COLAB:
    from google.colab import files
    files.download(archive_zip)
    print("Download started via Colab files.download")
elif ON_KAGGLE:
    print("Kaggle: open the Files panel and download this .zip manually.")
else:
    print("Vast.ai/local: download the .zip from your workspace or Jupyter file browser.")
